In [ ]:
#読み込みモデルの選択用
import os
read_name="C:/Users/i6root/Tkt/New study/2DCNN_results/2DCNN_Ct_fineTune-NN2/Adam/allequalver/train_3796_rate_0.0001_batch_16_epochs_300_optimizer_Adam/"
print(read_name)
os.path.exists(read_name)

In [ ]:
from tqdm import tqdm
import time
import cv2
import numpy as np
from keras.utils import np_utils
from PIL import Image
from keras import backend as K
from plot2 import Confusion_matrix
from keras.preprocessing.image import array_to_img

In [ ]:
from keras.models import model_from_json 
#json_string = open(os.path.join(read_name+'modelsmodel.json')).read()
json_string = open(os.path.join(read_name+'modelsmodel.json')).read()
model = model_from_json(json_string)
print(model)

In [ ]:
model.load_weights(read_name+'modelsweight.hdf5')
print(model)

In [ ]:
# 画像の読み込み
def label_value(serch):
    if "acceptable" in serch:
        value=0
    else :
        value=1
        
    return value

import csv
#データの読み込みのために、csv読み込み
def read_path_and_tag(csv_f):
    train=[]
    train_label=[]
    train_tag=[]
    valid=[]
    valid_label=[]
    valid_tag=[]
    test=[]
    test_label=[]
    test_tag=[]

    with open(csv_f) as f:
        reader =csv.reader(f)
        for row in reader:
            #print(row[0],row[1])
            #print(row[0])


            if row[1] == "test":
                test.append(row[0])
                test_label.append(label_value(row[0]))
                test_tag.append(row[1])
                #print(' ///test ')

            elif row[1]=="valid":
                valid.append(row[0])
                valid_label.append(label_value(row[0]))
                valid_tag.append(row[1])
                #print(' ///valid')

            elif row[1]=="train":
                train.append(row[0])
                train_label.append(label_value(row[0]))
                train_tag.append(row[1])
                #print(' ///train')

    return train,train_label,train_tag,valid,valid_label,valid_tag,test,test_label,test_tag


# CTace_path =  read_path_and_tag("file_way_CT_1of2.csv")
# CTcol_path =  read_path_and_tag("file_way_CT_2of2.csv")
#CTace_path =  read_path_and_tag("file_way_CT_1of2_headpart.csv")
#CTcol_path =  read_path_and_tag("file_way_CT_2of2_headpart.csv")
CTace_path =  read_path_and_tag("file_way_CT_1of2_2DForUse_retable.csv")
CTcol_path =  read_path_and_tag("file_way_CT_2of2_2DForUse_retable.csv")

array_name = ["train","train_label","train_tag","valid","valid_label","valid_tag","test","test_label","test_tag"]
array = ["train","train_label","train_tag","valid","valid_label","valid_tag","test","test_label","test_tag"]

for i in range(9):
    array[i] = CTace_path[i]+CTcol_path[i]
    
train = array[0]
train_label = array[1]
train_tag = array[2]
valid = array[3]
valid_label = array[4]
valid_tag = array[5]
test = array[6]
test_label = array[7]
test_tag = array[8]

for i in range(9):
    print(array_name[i],len(array[i]))
# print(train_label)

In [ ]:
#画像サイズ
size=224
num_class=2

In [ ]:
test_0=[]
test_1=[]

for i in test_label:
    if i==0:
        test_0.append(i)
    elif i==1:
        test_1.append(i)
        
print(len(test_0),len(test_1))

In [ ]:
#testも同数に
test_0=[]
test_1=[]
pickup_test=[]
test_equal=[]
random.seed(1)

for i in test:
    if "acceptable" in i:
        test_0.append(i)
    else:
        test_1.append(i)

if len(test_0)<len(test_1):
    pickup_test=random.sample(test_1,len(test_0))
    test_equal=test_0+pickup_test      
elif len(test_1)<len(test_0):
    pickup_test=random.sample(test_0,len(test_1))
    test_equal=test_1+pickup_test 

# test_equalのラベル作成
test_equal_label=[]
for i in test_equal:
    if "acceptable" in i:
        test_equal_label.append(0)
            
    else:
        test_equal_label.append(1)
            
test=[]
test_label=[]
for i in range(len(test_equal_label)):
    test.append(test_equal[i])
    test_label.append(test_equal_label[i])
print(len(test),len(test_label))

In [ ]:
test_img = []
for img in test:
    test_img.append(cv2.imread(img))
    
print(len(test_img))

In [ ]:
for i in range(len(test_img)):
    test_img[i] = cv2.resize(test_img[i],(224,224))

In [ ]:
print(type(test_img[0]))
print(test_img[0].shape)

In [ ]:
#画像の表示　内容の確認
from matplotlib import pyplot as plt
for i in range(4):
    plt.subplot(2,4,i+1)
    plt.imshow(test_img[i])

In [ ]:
learn_start=time.time()

In [ ]:
# Grad_Cam
def Grad_Cam(model, x, label,layer_name,size,save_name,view_name):
    K.set_learning_phase(1)
    X = np.expand_dims(x, axis=0)
    predictions = model.predict(X)
    pred = np.argmax(predictions)
    class_idx = np.argmax(predictions[0])

    class_output = model.output[:, class_idx]        
    conv_output = model.get_layer(layer_name).output   # layer_nameのレイヤーのアウトプット
    grads = K.gradients(class_output, conv_output)[0]  # gradients(loss, variables) で、variablesのlossに関しての勾配を返す
    gradient_function = K.function([model.layers[0].input, K.learning_phase()], [conv_output, grads])  # model.inputを入力すると、conv_outputとgradsを出力する関数

    output, grads_val = gradient_function([X])
    output, grads_val = output[0], grads_val[0]

    # 重みを平均化して、レイヤーのアウトプットに乗じる
    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)
    cam = cv2.resize(cam, (X.shape[2], X.shape[1]), cv2.INTER_LINEAR) # ブロック
    # cam = cv2.resize(cam, (x.shape[2], x.shape[1]))  # いい感じ

    cam = np.maximum(cam, 0)  # マイナス削除
    cam = cam / cam.max()  # 正規化

    jetcam = cv2.applyColorMap(np.uint8(size*cam), cv2.COLORMAP_JET)  # モノクロ画像に疑似的に色をつける
    jetcam = cv2.cvtColor(jetcam, cv2.COLOR_BGR2RGB)  # 色をRGBに変換
    jetcam = np.clip(np.float32(jetcam) + X[0]*255*0.5, 0, 255)# もとの画像に合成
    jetcam = np.uint8(jetcam)
    jetcam.reshape(X.shape[1:])
    
    if pred==1 and label==1:
        Class="tp"
        
    if pred==1 and label==0:
        Class="fp"
        
    if pred==0 and label==0:
        Class="tn"
        
    if pred==0 and label==1:
        Class="fn"
    
    #色付けした画像の保存
    os.makedirs(save_name+'GradCam_folder'+'/'+'number'+str(i+1)+'/',exist_ok=True)
    save_img=array_to_img(jetcam)#
    plt.figure()
    plt.title(save_view_name+str(i+1)+'Grad_Cam'+"\n"+"==result:"+Class,fontsize=17)
    plt.imshow(save_img)
    
    plt.savefig(save_name+'GradCam_folder'+'/'+'number'+str(i+1)+'/'+str(i+1)+'Grad_Cam.png',bbox_inches='tight')#  Grad_Camの保存
    plt.close
    os.makedirs(save_name+'GradCam_folder'+'/'+'number'+str(i+1)+'/',exist_ok=True)
    plt.figure()
        
    plt.title(save_view_name+str(i+1)+'Original_test_img'+"\n"+"==result:"+Class,fontsize=17)
    plt.imshow(x)
    plt.savefig(save_name+'GradCam_folder'+'/'+'number'+str(i+1)+'/'+str(i+1)+'Original_test_img.png',bbox_inches='tight')#  Grad_Camの保存
    plt.close    
    return print('finish')


In [ ]:
from matplotlib import pyplot as plt
save_view_name="Grad_Cam_result"
#layer_folder=["conv1_pad","conv1","bn_conv1","pool1_pad"]
# layer_folder=["res2a_branch2a","bn2a_branch2a","res2a_branch2b","bn2a_branch2b","res2a_branch2c","res2a_branch1","bn2a_branch2c","bn2a_branch1"]
# layer_folder=["res2b_branch2a","bn2b_branch2a","res2b_branch2b","bn2b_branch2b","res2b_branch2c","bn2b_branch2c"]
# layer_folder=["res2c_branch2a","bn2c_branch2a","res2c_branch2b","bn2c_branch2b","res2c_branch2c","bn2c_branch2c"]
# layer_folder=["res3a_branch2a","bn3a_branch2a","res3a_branch2b","bn3a_branch2b","res3a_branch2c","res3a_branch1","bn3a_branch2c","bn3a_branch1"]
# layer_folder=["res3b_branch2a","bn3b_branch2a","res3b_branch2b","bn3b_branch2b","res3b_branch2c","bn3b_branch2c"]
# layer_folder=["res3c_branch2a","bn3c_branch2a","res3c_branch2b","bn3c_branch2b","res3c_branch2c","bn3c_branch2c"]
# layer_folder=["res3d_branch2a","bn3d_branch2a","res3d_branch2b","bn3d_branch2b","res3d_branch2c","bn3d_branch2c"]
# layer_folder=["res4a_branch2a","bn4a_branch2a","res4a_branch2b","bn4a_branch2b","res4a_branch2c","res4a_branch1","bn4a_branch2c","bn4a_branch1"]
# layer_folder=["res4b_branch2a","bn4b_branch2a","res4b_branch2b","bn4b_branch2b","res4b_branch2c","bn4b_branch2c"]
# layer_folder=["res4c_branch2a","bn4c_branch2a","res4c_branch2b","bn4c_branch2b","res4c_branch2c","bn4c_branch2c"]
# layer_folder=["res4d_branch2a","bn4d_branch2a","res4d_branch2b","bn4d_branch2b","res4d_branch2c","bn4d_branch2c"]
# layer_folder=["res4e_branch2a","bn4e_branch2a","res4e_branch2b","bn4e_branch2b","res4e_branch2c","bn4e_branch2c"]
#layer_folder=["res4f_branch2a","bn4f_branch2a","res4f_branch2b","bn4f_branch2b","res4f_branch2c","bn4f_branch2c"]
# layer_folder=["res5a_branch2a","bn5a_branch2a","res5a_branch2b","bn5a_branch2b","res5a_branch2c","res5a_branch1","bn5a_branch2c","bn5a_branch1"]
# layer_folder=["res5b_branch2a","bn5b_branch2a","res5b_branch2b","bn5b_branch2b","res5b_branch2c","bn5b_branch2c"]
#layer_folder=["res5c_branch2a","bn5c_branch2a","res5c_branch2b","bn5c_branch2b","res5c_branch2c","bn5c_branch2c"]
layer_folder=["bn3d_branch2c"]
size=224
for roop_count in tqdm(range(len(layer_folder))):
    layer=layer_folder[roop_count]
    save_name="C:\\Users\\i6root\\Tkt\\Grad_Cam_programs\\Grad_Cam_results\\20221206_results\\finetune\\"+layer+"\\"
    for i in tqdm(range(len(test_img))):
        Grad_Cam(model, test_img[i],test_label[i],layer,size,save_name,save_view_name)

In [ ]:
# color="Greens"
# p=Confusion_matrix(model,test_img,test_label,save_name,color,save_view_name)

In [ ]:
learn_time=time.time()-learn_start
learn_time/=60
print("{:.04}min in learn".format(learn_time))
print("finish")